In [ ]:
import itertools
import math
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

pd.set_option('display.max_column', None)

# Context

In [ ]:
data = pd.read_csv("../data/train.csv")
# Notice: embarked 	Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton
# Notice: sibsp 	# of siblings / spouses aboard the Titanic
# Notice: parch 	# of parents / children aboard the Titanic

# Data quality assessment

In [ ]:
data.info()

In [ ]:
assert data['PassengerId'].is_unique
assert 0 == data[~data['Survived'].isin((0, 1))].size
data['Survived'] = data['Survived'].astype('bool')
assert 0 == data[~data['Embarked'].isin(['C', 'Q', 'S', np.nan])].size
assert 0 == data[~data['Pclass'].isin([1, 2, 3])].size
# Проверим возможные полные дубликаты.
assert 0 == data[data.duplicated(subset=list(filter(lambda c: c != 'PassengerId', data.columns)))].size
# И разберём сложные  строки.
data[['TicketPref', 'TicketNum']] = data['Ticket'].str.extract(r'(?:(.+)\s)?(\d+)')
# Нулевых билетов нет, поэтому можно заменить NaN на 0
# TODO: data[data['TicketNum'].isnull()] = 0
data['TicketNum'] = data['TicketNum'].astype('float64')
data[['CabinPref', 'CabinNum']] = data['Cabin'].str.extract(r'([A-Za-z])(\d+)')
# Нулевых кают нет, поэтому можно заменить NaN на 0
# TODO: data[data['CabinNum'].isnull()] = 0
data['CabinNum'] = data['CabinNum'].astype('float64')

Поскольку здесь нет полностью бесполезных строк, ничего удалять не будем. При необходимости данные будем фильтровывать, исключая кортежи с пустыми значениями.

# Data exploration

## Отдельные параметры

In [ ]:
data.describe()

Явно аномальных значений (например, возраст в 1000 лет) нет.

In [ ]:
corr_b_fields = ['Survived']
corr_n_fields = ['Pclass', 'Age', 'SibSp', 'Fare', 'TicketNum', 'CabinNum']
corr_o_fields = ['Sex', 'TicketPref', 'CabinPref', 'Embarked']

for f in corr_o_fields:
    plt.subplots()
    sns.countplot(data=data[~data[f].isnull()], x=f)
for f in corr_n_fields:
    plt.subplots()
    sns.histplot(data=data[~data[f].isnull()], x=f)

Здесь речь о тех пассажирах для которых известен параметр.
На данный момент префиксы билетов и кают ничего не дают. Числовая часть билетов явно имеет повторы, причины неизвестны. Стоимость проезда имеет три выраженных пика, вероятно, связана с пассажирским классом. Интересны пики в гистограмме возраста, стоит рассмотреть связи. Наибольшее количество пассажиров в диапазоне 20-30 лет. Больше всего пассажиров зашло на борт в Саутгемптоне. Преобладающий пол мужской, класс - третий.

In [ ]:
#data.groupby('Age').count().sort_values(by='PassengerId', ascending=False)

In [ ]:
data.corr(method="pearson")

Наблюдаем слабую связь выживаемости с возрастом (0.103895), стоимостью билета (0.187534) и среднюю связь с пассажирским классом (-0.289723).
Стоимость билета сильно связана с пассажирским классом (очевидно, 1-й дороже).

In [ ]:
for f1, f2 in itertools.pairwise(corr_o_fields):
    plt.subplots()
    sns.heatmap(data=data, x=f1, y=f2)


## Бонус. Спасают ли от утопления "счастливые билеты"?

In [ ]:
lucky_data = data[~data['TicketNum'].isnull()][['Survived', 'TicketNum']]
def is_lucky(n):
    n = str(int(n))
    return sum(map(int, n[:math.floor(len(n)/2)])) == sum(map(int, n[math.ceil(len(n)/2):]))

lucky_data['is_lucky'] = lucky_data['TicketNum'].apply(is_lucky)
lucky_alive = lucky_data[lucky_data['is_lucky'] & lucky_data['Survived']].count().values[0] / lucky_data[lucky_data['is_lucky']].count().values[0]
unlucky_alive = lucky_data[~lucky_data['is_lucky'] & lucky_data['Survived']].count().values[0] / lucky_data[~lucky_data['is_lucky']].count().values[0]
print(f"Выживших со счастливым билетом {int(lucky_alive*100)}%, без счастливого {int(unlucky_alive*100)}%.")

# Summary